In [22]:
import os
import re
import time

import pandas as pd
from pandas import DataFrame
from openpyxl import load_workbook
import requests

In [23]:
from util.widgets import generate_ym, generate_ymd, generate_i

ym = generate_ym()
g = 'cargohose_herren'
bd = f'{g}/{ym}'
b_d = f'{g}_{ym}'

os.makedirs(bd, exist_ok=True)

ymd = generate_ymd()

fp_tgt = f'{g}/{g}_tgt.xlsx'

In [24]:
tgt_df: DataFrame = pd.read_excel(fp_tgt, sheet_name='tgt')
from util.widgets import Status, note_inuse_sku, note_archived_sku, note_deleted_sku, note_recalled_sku, note_other_sku
tgt_df['status'] = Status.UNUSED.value

tgt_df = note_inuse_sku(tgt_df)
tgt_df = note_archived_sku(tgt_df)
tgt_df = note_deleted_sku(tgt_df)
tgt_df = note_recalled_sku(tgt_df)
tgt_df = note_other_sku(tgt_df)
tgt_df

,sf,fp,imageUrl,productSku,amazon_psku,ASIN,note,productLabelList,categoryName,warehouseName,...,buyerName,vectorDistance,buyingPrice,material,withholdStock,productSkuHref,similarity,ymd,status,ymd_HMS
0,default,cargohose_herren/2024_07_22_(08-09)/img/defaul...,http://183.62.143.166:28099/image/product/ftp/...,FBE230705004,FRL-DE-24-07-26-15-52-25-001,NaN,主图背景色,"['定制', '可压缩产品', '新版', '已换图']",Men's Casual Pants 男士休闲裤,顺德仓库,...,邓烨萍,0.3605,26.0,Polyester,0.0,/links/erp/products?query.productSku=FBE230705004,0.6395,2024_07_22,InUse,2024-07-26 15:53:25
1,default,cargohose_herren/2024_07_22_(08-09)/img/defaul...,http://183.62.143.166:28099/image/product/ftp/...,LJH90722333,NaN,NaN,NaN,"['可压缩产品', '部分停产下架', '部分缺货']",Men's Casual Pants 男士休闲裤,顺德仓库,...,罗智斌,0.4822,34.5,Polyester,0.0,/links/erp/products?query.productSku=LJH90722333,0.5178,2024_07_22,Archived,NaN
2,default,cargohose_herren/2024_07_22_(08-09)/img/defaul...,http://183.62.143.166:28099/image/product/ftp/...,LBB230904033,NaN,NaN,NaN,"['可压缩产品', '冬季', '一件起订', '部分停产下架']",Men's Casual Pants 男士休闲裤,顺德仓库,...,罗智斌,0.4943,19.0,Polyester,0.0,/links/erp/products?query.productSku=LBB230904033,0.5057,2024_07_22,Archived,NaN
3,default,cargohose_herren/2024_07_22_(08-09)/img/defaul...,http://183.62.143.166:28099/image/product/ftp/...,FZQ240108001,FRL-DE-24-07-26-15-52-25-002,NaN,NaN,['冬季'],Men's Casual Pants 男士休闲裤,顺德仓库,...,胡欣,0.3388,33.0,Polyester,0.0,/links/erp/products?query.productSku=FZQ240108001,0.6612,2024_07_22,InUse,2024-07-26 15:53:25
4,default,cargohose_herren/2024_07_22_(08-09)/img/defaul...,http://183.62.143.166:28099/image/product/ftp/...,LBB230904025,NaN,NaN,NaN,"['可压缩产品', '冬季', '一件起订', '部分停产下架']",Men's Casual Pants 男士休闲裤,顺德仓库,...,罗智斌,0.5005,19.0,Polyester,0.0,/links/erp/products?query.productSku=LBB230904025,0.4995,2024_07_22,Archived,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241,best,cargohose_herren/2024_07_22_(08-09)/img/best/c...,http://183.62.143.166:28099/image/product/ftp/...,CJN211214321,NaN,NaN,NaN,"['可压缩产品', '部分停产下架']",Women's Casual Pants 女士休闲裤,顺德仓库,...,杨汉玉,0.3919,26.0,Cotton,0.0,/links/erp/products?query.productSku=CJN211214321,0.6081,2024_07_22,Unused,NaN
1242,best,cargohose_herren/2024_07_22_(08-09)/img/best/c...,http://183.62.143.166:28099/image/product/ftp/...,ZRR240624003,NaN,NaN,NaN,[],Men's Cargo Pants 男士工装裤,顺德仓库,...,郑楚薇,0.2931,65.0,"Nylon,Spandex",1.0,/links/erp/products?query.productSku=ZRR240624003,0.7069,2024_07_22,Unused,NaN
1243,best,cargohose_herren/2024_07_22_(08-09)/img/best/c...,http://183.62.143.166:28099/image/product/ftp/...,HYM231006008,NaN,NaN,NaN,['可压缩产品'],Men's Casual Pants 男士休闲裤,顺德仓库,...,罗智斌,0.3094,37.0,"Nylon,Spandex",2.0,/links/erp/products?query.productSku=HYM231006008,0.6906,2024_07_22,Unused,NaN
1244,best,cargohose_herren/2024_07_22_(08-09)/img/best/c...,http://183.62.143.166:28099/image/product/ftp/...,XJJ221028015,NaN,NaN,NaN,"['圣诞节产品', '可压缩产品', '部分停产下架']",Women's Casual Pants 女士休闲裤,顺德仓库,...,罗智斌,0.3265,21.0,Polyester,0.0,/links/erp/products?query.productSku=XJJ221028015,0.6735,2024_07_22,Unused,NaN


In [25]:
# 将修改后的DataFrame写回到Excel文件中的特定工作簿
with pd.ExcelWriter(fp_tgt) as writer:
    tgt_df.to_excel(writer, index=False, sheet_name='tgt')